In [1]:
import json
import pathlib

DATA_DIR = pathlib.Path('../data')

In [2]:
def load_data(data_dir):
    data = []
    for filepath in data_dir.glob('*.json'):
        with open(filepath) as f:
            data.extend(json.load(f))
    return data

In [3]:
data = load_data(DATA_DIR)

In [4]:
print(data[0]['board'])

11111111111
1         1
1         1
1         1
1         1
1    1    1
1         1
1         1
1         1
1         1
11111111111



In [5]:
data[0]['ticks']

12